In [30]:
# Transformers and PyTorch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu116
!pip install transformers==4.35.0

# CLIP
!pip install git+https://github.com/openai/CLIP.git

# FAISS GPU (if you have CUDA)
!pip install faiss-gpu
# FAISS CPU alternative:
# pip install faiss-cpu

# PIL for image handling
!pip install pillow

# Other utilities
!pip install jsonlines numpy


Looking in indexes: https://download.pytorch.org/whl/cu116
  Using cached transformers-4.35.0-py3-none-any.whl.metadata (123 kB)
  Using cached tokenizers-0.14.1-cp38-none-win_amd64.whl.metadata (6.8 kB)
  Using cached huggingface_hub-0.17.3-py3-none-any.whl.metadata (13 kB)
Using cached transformers-4.35.0-py3-none-any.whl (7.9 MB)
Using cached tokenizers-0.14.1-cp38-none-win_amd64.whl (2.2 MB)
Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.35.3
    Uninstalling huggingface-hub-0.35.3:
      Successfully uninstalled huggingface-hub-0.35.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.

  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\sarav\AppData\Local\Temp\pip-req-build-q44imrmi'
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [ ]:
!conda install -c conda-forge cudatoolkit=11.2 faiss-gpu-y

3 channel Terms of Service accepted
Channels:
 - defaults
 - conda-forge/label/broken
Platform: win-64
Solving environment: failed



LibMambaUnsatisfiableError: Encountered problems while solving:
  - nothing provides cudatoolkit 10.2|10.2.* needed by faiss-1.7.0-py36cuda102hfea3036_2_cuda

Could not solve for environment specs
The following package could not be installed
└─ faiss-gpu =* * is not installable because it requires
   └─ faiss ==1.7.0 *_cuda but there are no viable options
      ├─ faiss 1.7.0 would require
      │  └─ cudatoolkit ==10.2|=10.2 *, which does not exist (perhaps a missing channel);
      ├─ faiss 1.7.0 would require
      │  └─ cudatoolkit ==11.0|=11.0 *, which does not exist (perhaps a missing channel);
      ├─ faiss 1.7.0 would require
      │  └─ cudatoolkit ==11.1|=11.1 *, which does not exist (perhaps a missing channel);
      └─ faiss 1.7.0 would require
         └─ cudatoolkit ==11.2|=11.2 *, which does not exist (perhaps a missing channel).



In [32]:
!pip install h5py

In [1]:
import torch
torch.cuda.empty_cache()
import faiss
import json
import numpy as np
from PIL import Image
from transformers import (
    AutoTokenizer, AutoModel, AutoConfig, AutoModelForCausalLM
)
import clip
import time

# --- Custom imports from your repo ---
from src.utils import prep_strings, postprocess_preds
from src.gpt2 import ThisGPT2Config, ThisGPT2LMHeadModel
from src.vision_encoder_decoder import SmallCap, SmallCapConfig



d:\anaconda\envs\smallcap-clean\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ["HF_HOME"] = "C:/Users/sarav/.cache/huggingface"


In [4]:
!pip install hnswlib

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp38-cp38-win_amd64.whl size=158395 sha256=ed7f035026b0ad3447d538d155ae4931db130f393300da6266fc699f608040b3
  Stored in directory: c:\users\sarav\appdata\local\pip\cache\wheels\c9\52\92\b4b22fa3e652073584067109b5094dbd5c73b42738e41213f6
Successfully built hnswlib


In [ ]:
import torch
import json
import numpy as np
from PIL import Image
import time
import faiss
import clip
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForCausalLM

# --- Force CPU ---
device = "cpu"

print("Loading GPT2")
# --- Load and configure tokenizer ---
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = "!"
tokenizer.eos_token = "."

# --- Register custom models ---
AutoConfig.register("this_gpt2", ThisGPT2Config)
AutoModel.register(ThisGPT2Config, ThisGPT2LMHeadModel)
AutoModelForCausalLM.register(ThisGPT2Config, ThisGPT2LMHeadModel)

AutoConfig.register("smallcap", SmallCapConfig)
AutoModel.register(SmallCapConfig, SmallCap)

print("Loading SmallCap")
# --- Load model on CPU ---
model = AutoModel.from_pretrained("Yova/SmallCap7M", torch_dtype=torch.float16,low_cpu_mem_usage=True  ).to(device)

print("Loading CLIP")
# Retrieval model also on CPU
retrieval_model, preprocess_retrieval = clip.load("RN50", device="cpu")
feature_extractor = clip.load("ViT-B/32", device="cpu")[1]

# --- Load prompt template ---
with open("src/template.txt", "r") as f:
    template = f.read().strip()

# --- Load COCO captions & FAISS index ---
captions = json.load(open("datastore/coco_index_captions.json"))
retrieval_index = faiss.read_index("datastore/coco_index")  # stays on CPU

# --- Retrieval helper ---
def retrieve_caps(image_embedding, index, k=4):
    xq = image_embedding.astype(np.float32)
    faiss.normalize_L2(xq)
    D, I = index.search(xq, k)
    return I

# --- Load example image ---
image_path = r"E:\Projects\smallcap\data\image_truck.jpg"
image = Image.open(image_path).convert("RGB")

# --- Retrieve captions ---
start = time.time()
image_preprocessed = preprocess_retrieval(image).unsqueeze(0).to("cpu")
with torch.no_grad():
    image_embedding = retrieval_model.encode_image(image_preprocessed).cpu().numpy()

nns = retrieve_caps(image_embedding, retrieval_index)[0]
caps = [captions[i] for i in nns][:4]

# --- Prepare prompt ---
decoder_input_ids = prep_strings(
    "",
    tokenizer,
    template=template,
    retrieved_caps=caps,
    k=4,
    is_test=True,
)

# --- Generate caption on CPU ---
pixel_values = feature_extractor(image, return_tensors="pt").pixel_values
with torch.no_grad():
    pred = model.generate(
        pixel_values.to("cpu"),
        decoder_input_ids=torch.tensor([decoder_input_ids]).to("cpu"),
        max_new_tokens=25,
        no_repeat_ngram_size=0,
        length_penalty=0,
        min_length=1,
        num_beams=3,
        eos_token_id=tokenizer.eos_token_id,
    )

print(f"Time taken: {time.time() - start:.2f} seconds\n")
image.show()

print("Prompt:\n")
print(tokenizer.decode(decoder_input_ids))

print("\nGenerated caption:\n")
print(postprocess_preds(tokenizer.decode(pred[0]), tokenizer))



Loading GPT2
Loading SmallCap


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 9437184 bytes.

In [ ]:
from sentence_transformers import SentenceTransformer

# --- Device setup ---
device = "cuda" if torch.cuda.is_available() else "cpu"

# --- Load SmallCap / GPT2 model ---
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = '!'
tokenizer.eos_token = '.'

AutoConfig.register("smallcap", SmallCapConfig)
AutoModel.register(SmallCapConfig, SmallCap)
model = AutoModel.from_pretrained("Yova/SmallCap7M", torch_dtype=torch.float16).to(device)

# --- Load template ---
with open("src/template.txt") as f:
    template = f.read().strip()

# --- Load captions ---
import json
captions = json.load(open('datastore/coco_index_captions.json'))

# --- Load SentenceTransformer (CLIP) ---
retrieval_model = SentenceTransformer("sentence-transformers/clip-ViT-B-32", device=device)

# --- Precompute caption embeddings (do once, reuse) ---
caption_embeddings = retrieval_model.encode(captions, convert_to_numpy=True, normalize_embeddings=True)

# --- Load example image ---
image_path = rf"E:\Projects\smallcap\data\image_truck.jpg"
image = Image.open(image_path).convert("RGB")

# --- Encode image for retrieval ---
# You can either generate a textual description or use a placeholder
# If you want full image embeddings, integrate CLIP image encoder here
image_description = "A photo of the image"  # placeholder
image_embedding = retrieval_model.encode([image_description], convert_to_numpy=True, normalize_embeddings=True)

# --- Retrieve top-4 captions ---
cosine_sim = np.dot(image_embedding, caption_embeddings.T)  # shape: (1, num_captions)
topk_idx = np.argsort(-cosine_sim[0])[:4]
retrieved_caps = [captions[i] for i in topk_idx]

print("Retrieved captions:", retrieved_caps)

# --- Prepare prompt ---
decoder_input_ids = prep_strings(
    '', tokenizer, template=template, retrieved_caps=retrieved_caps, k=4, is_test=True
)

# --- Generate caption ---
pixel_values = torch.zeros((1, 3, 224, 224))  # placeholder if SmallCap requires image input
start = time.time()
with torch.no_grad():
    pred = model.generate(
        pixel_values.to(device),
        decoder_input_ids=torch.tensor([decoder_input_ids]).to(device),
        max_new_tokens=25,
        no_repeat_ngram_size=0,
        length_penalty=0,
        min_length=1,
        num_beams=3,
        eos_token_id=tokenizer.eos_token_id
    )

print(f"Time taken: {time.time() - start:.2f} seconds\n")
display(image)

print("Prompt:\n")
print(tokenizer.decode(decoder_input_ids))

print("\nGenerated caption:\n")
print(postprocess_preds(tokenizer.decode(pred[0]), tokenizer))



ImportError: cannot import name 'get_token' from 'huggingface_hub.utils' (d:\anaconda\envs\smallcap-clean\lib\site-packages\huggingface_hub\utils\__init__.py)